In [1]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/postgresql-42.2.8.jar pyspark-shell'

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("retailrocket").getOrCreate()

url = "jdbc:postgresql://db/postgres"
properties = {
    "driver": "org.postgresql.Driver",
    "user": "postgres",
    "password": "retailrocket"
}

df = spark.read.jdbc(url=url,\
    table="category_tree", \
    properties=properties)
df.show()

+----------+--------+
|categoryid|parentid|
+----------+--------+
|      1016|     213|
|       809|     169|
|       570|       9|
|      1691|     885|
|       536|    1691|
|       231|    null|
|       542|     378|
|      1146|     542|
|      1140|     542|
|      1479|    1537|
|        83|    1621|
|       688|     893|
|       257|     312|
|      1640|     622|
|       963|    1281|
|       412|    1110|
|       948|    1110|
|       934|    1110|
|       148|    1110|
|        12|    1110|
+----------+--------+
only showing top 20 rows

